In [31]:
from shapely import Point, LineString, Polygon
import geopandas as gpd

test = None

def get_visibility_result(point, buildings, view_distance):
    global test
    crs = buildings.crs
    point_buffer = Point(point).buffer(view_distance,resolution=64)
    
    s = buildings.within(point_buffer)
    
    buildings_in_buffer = buildings.loc[s[s].index].reset_index(drop=True)
    
    test1 = [Point(geom.exterior.coords) for geom in buildings_in_buffer.geometry.unary_union.geoms]
    print(test1)
    buffer_exterior_ = list(point_buffer.exterior.coords)
    line_geometry = [LineString([point, ext]) for ext in buffer_exterior_]
    buffer_lines_gdf = gpd.GeoDataFrame(geometry=line_geometry,crs=crs)
    
    united_buildings = buildings_in_buffer.unary_union

    if united_buildings:
        splited_lines = buffer_lines_gdf['geometry'].apply(lambda x: x.difference(united_buildings))
    else:
        splited_lines = buffer_lines_gdf["geometry"]
    
    splited_lines_gdf = gpd.GeoDataFrame(geometry=splited_lines).explode(index_parts=True)
    
    splited_lines_list = []

    for u, v in splited_lines_gdf.groupby(level=0):
        splited_lines_list.append(v.iloc[0]['geometry'].coords[-1])

    circuit = Polygon(splited_lines_list)
    
    
    if united_buildings:
        circuit = circuit.difference(united_buildings)
    
    view_zone = gpd.GeoDataFrame(geometry=[circuit]).set_crs(crs)
    return view_zone

builds = gpd.read_file('examples_data/tara_buildings.geojson').to_crs(3857)
p = Point(8278652.47 ,7737142.18) 
point = gpd.GeoDataFrame(geometry=[p],crs=3857)
view_zone = get_visibility_result(p, builds, 300)

m1 = view_zone.explore()
m2=point.explore(m=m1,color='red')
m3 =builds.explore(m=m2,color='purple')
# test.explore(m=m3,color='yellow')
m3

TypeError: 'MultiPolygon' object is not iterable